In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install openai==0.28.0
!pip install huggingface_hub
!pip install groq
!pip instrall requests

ERROR: unknown command "instrall" - maybe you meant "install"


In [ ]:
import os
from groq import Groq
from huggingface_hub import InferenceClient
import pandas as pd
import json
from google.colab import userdata

In [ ]:
import os
import google.colab
from huggingface_hub import InferenceClient, login, HfFolder
from transformers import AutoTokenizer


hf_token = os.getenv("HF_TOKEN")


if hf_token and not HfFolder.get_token():
    login(token=hf_token)


hf_tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B", token=hf_token) #tokenizer

#truncate the prompt
def truncate_prompt(prompt, max_input_tokens=3900):
    input_ids = hf_tokenizer(prompt, return_tensors="pt").input_ids[0]
    if len(input_ids) > max_input_tokens:
        prompt = hf_tokenizer.decode(input_ids[:max_input_tokens], skip_special_tokens=True)
    return prompt


In [ ]:
import os
import time
import requests
import google.colab
class Agent:
    def __init__(self, name, role, llm_type):
        self.name = name
        self.role = role
        self.llm_type = llm_type.lower()
        self.memory = [] #memory for each agent is stored

        #initialization of hugging face client
        if self.llm_type == "hf":
            self.hf_client = InferenceClient("google/flan-t5-base") #google flan or 'microsoft/Phi-3-mini-4k-instruct'

        elif self.llm_type == 'groq':
          self.groq_api_key = "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"
          self.groq_url = "https://api.groq.com/openai/v1/chat/completions"


    def add_to_memory(self, prompt, response):
      self.memory.append({"prompt": prompt, "response": response})

    def get_context(self, window_size=2): #remembers 2 converstaions
      if not self.memory:
        return ""  # is empty if there is no memory yet
      context = ""
      for item in self.memory[-window_size:]:
          context += f"User: {item['prompt']}\n{self.name}: {item['response']}\n"
      return context



    def generate_response(self, prompt):
        context = self.get_context()
        full_prompt = context + f"User: {prompt}\n{self.name}:" #total prompt including memory

        if self.llm_type == 'hf':
          response = self.query_huggingface(full_prompt)

        elif self.llm_type == 'groq':
          response = self.query_groq(full_prompt)
          time.sleep(3)


        else:
            response = f"{self.name} ({self.role}): Unsupported LLM type."

        self.add_to_memory(prompt, response)
        return response





    def query_huggingface(self, prompt):
      try:

        prompt = truncate_prompt(prompt, max_input_tokens=2048) #truncate the prompt
        return self.hf_client.text_generation(prompt, max_new_tokens=300)
      except Exception as e:

        #if failed,try with a smaller truncated prompt
        try:
            return "[HuggingFace Error - retrying with shorter prompt] " + \
                   self.hf_client.text_generation(prompt[:300], max_new_tokens=30)
        except Exception as e2:
            return f"[HF Final Error] {str(e2)}"




    def query_groq(self, prompt):
      try:

          if len(prompt) > 7000 :
            prompt = prompt[-7000:] #take 7000
          headers = {
            'Authorization': f'Bearer {self.groq_api_key}',
            'Content-Type': 'application/json'
          }
          messages = [
            {"role": "system", "content": f"You are {self.name}, a {self.role} in a courtroom. Respond accordingly."},
            {"role": "user", "content": prompt}
          ]
          payload = {
              "model": "llama-3.3-70b-versatile",
              "messages": messages,
              "max_tokens": 300,
              "temperature": 0.3,
              "top_p" : 0.8,
              "stream" : False
          }
          response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers=headers, json=payload)
          response_json = response.json()
          if response.status_code == 200:
            return response_json['choices'][0]['message']['content'].strip()

          elif response.status_code == 429:  # rate limit

            if "Retry-After" in response.headers:
                wait_time = float(response.headers["Retry-After"])
            else:
                wait_time = 6  # fallback
            print(f"[Rate Limit] Waiting {wait_time} seconds...")
            time.sleep(wait_time)
            return self.query_groq(prompt)

          else:
            return f"[Groq Error] {response_json.get('error', 'Unknown error')}"
      except Exception as e:
          return f"[Groq Error] {str(e)}"


In [ ]:
class CourtroomAgents:
    def __init__(self):
        self.agents = {}

    def add_agent(self, name, role, llm_type):
        agent = Agent(name, role, llm_type)
        self.agents[role.lower()] = agent

    def get_agent(self, role):
        return self.agents.get(role.lower(), None)

    def simulate_opening_statements(self, case_summary, accusation):
        print(" Trial Opening Statements")

        judge = self.get_agent('judge')
        prosecutor = self.get_agent('prosecutor')
        defense = self.get_agent('defense lawyer')
        witness = self.get_agent('witness')
        defendent = self.get_agent('defendent')
        plaintiff = self.get_agent('plaintiff')
        # Judge opens court
        judge_prompt = f"You are a judge. Open the courtroom and summarize the case precisely without showing any bias in not a long way: {case_summary}"
        casetype =judge.generate_response(f"you have read the summary of the case : {case_summary} now tell what type of case it is(answer should be of the form (civil case) or (criminal case) in one word)")
        print(f"============== {casetype} ====================")
        print(f" Judge {judge.name}:")
        print(judge.generate_response(judge_prompt))

        # Prosecutor gives opening statement
        prosecutor_prompt = f"You are a prosecutor. Present the accusations against the defendant correctly and shortly.Be confident and aggressive and express in a short way : {accusation}"
        print(f"\n Prosecutor {prosecutor.name}:")
        print(prosecutor.generate_response(prosecutor_prompt))

        #plaintiff gives his side
        if "civil" in casetype.lower():
          plaintiff_prompt = f"You are the plaintiff. Express your distress to the judge in a short way: {case_summary}"
          print(f"\n Plaintiff {plaintiff.name}:")
          print(plaintiff.generate_response(plaintiff_prompt))


          # Defense gives opening statement
          defense_prompt = f"You are a defense lawyer. Begin your defense for the accused.Be calculative and answer more accurately and in a short way. Accusation: {accusation}"
          print(f"\n Defense Lawyer {defense.name}:")
          print(defense.generate_response(defense_prompt))

    def simulate_witness_testimony(self, witness_name, role, testimony_prompt):
        witness = self.get_agent("witness")
        print(f"\n {witness_name} ({role}) is called to testify.")
        testimony = witness.generate_response(f"based on the following text produce a testimony for the witness in a short way:\n\n'{testimony_prompt}'") #asking llm to observe a testimony in the text

        response = witness.generate_response(f"you need to Testify as a {role} on the following: {testimony}") #witness gives his testification
        print(f"\n{witness_name} testifies: {response}")

    def simulate_cross_examination(self, questioning_party, questioned_party, testimony):

      questioning_agent = self.get_agent(questioning_party)
      questioned_agent = self.get_agent(questioned_party)

      # Ask LLM to generate a cross-exam question based on the testimony
      question_prompt = f"Based on the following testimony, generate a critical cross-examination question:\n\n'{testimony}'"
      question = questioning_agent.generate_response(question_prompt)

      print(f"\n Cross-Examination by {questioning_party} of {questioned_party}")
      print(f"{questioning_party}: {question}")

      # questioned party responds to the question
      response = questioned_agent.generate_response(f"{questioning_party} asks: {question} generate a precise response")
      print(f"{questioned_party}: {response}")

    def simulate_closing_statements(self):
        jury = self.get_agent('jury')
        print("\n Closing Statements")

        judge = self.get_agent('judge')
        prosecutor = self.get_agent('prosecutor')
        defense = self.get_agent('defense lawyer')

        # Prosecutor closing statement
        prosecutor_prompt = "Make your closing argument and request the judge to take appropriate measures."
        print(f"\n Prosecutor {prosecutor.name}:")
        print(prosecutor.generate_response(prosecutor_prompt))

        # Defense closing statement
        defense_prompt = "Make your closing argument and request the judge to take appropriate measures."
        print(f"\n Defense Lawyer {defense.name}:")
        print(defense.generate_response(defense_prompt))

        # Judge gives the verdict
        judge_prompt = "Give the verdict on the case considering each and every detail in small and effective way."
        print(f"\n Judge {judge.name}:")
        print(judge.generate_response(judge_prompt))

        #jury gives the verdict
        jury_prompt = "learn about the whole case and inspect all details and tell whether the justice is done or not in a small way"
        print(f"\n Jury gives the statement: \n")
        print(jury.generate_response(jury_prompt))

    def predict_verdict(self, case_text):

        judge = self.get_agent('judge')
        if not judge:
            return "DENIED"  # fallback

        prompt = (
            "You are the judge. Read the following case summary and give the final verdict "
            "in one word: GRANTED or DENIED.\n\n"
            f"Case:\n{case_text}\n\nVerdict:"
        )

        try:
            response = judge.generate_response(prompt)
        except Exception as e:
            print(f"[LLM Error]: {e}")
            return "DENIED"

        if response is None:
            return "DENIED"

        return str(response).strip().upper()




In [ ]:
import pandas as pd
from tqdm import tqdm
import time
import sys

# Load the CSV file
file_path = '/content/drive/My Drive/cases.csv'
data = pd.read_csv(file_path)

# Select the first 50 cases
data = data.head(50)

# Print header for the output
print("id,label")

# Loop through each case
for i, row in tqdm(data.iterrows(), total=len(data), desc="Processing cases"):
    case_id = row['id'] if 'id' in row else i
    case_summary = row['text']

    # Create a new courtroom and add the judge
    court = CourtroomAgents()
    court.add_agent("Justice Ray", "Judge", "groq")

    # Call the 'predict_verdict' method to get the verdict for this case
    try:
        verdict = court.predict_verdict(case_summary)
        print(f"Raw response for case {case_id}: {verdict}")  # Debug
    except Exception as e:
        print(f"[Error for case {case_id}]: {str(e)}", file=sys.stderr)
        verdict = None  # Fallback for error

    # Make sure verdict is a string
    if isinstance(verdict, str):
        verdict = verdict.strip().upper()
    elif verdict is None:
        verdict = "DENIED"  # Default fallback
    else:
        verdict = str(verdict).strip().upper()

    # Convert final decision to label
    if "GRANTED" in verdict:
        label = 1
    elif "DENIED" in verdict:
        label = 0
    else:
        label = 0  # fallback

    # Print result in required format
    print(f"{case_id},{label}")

    # Sleep to avoid rate limits
    time.sleep(5)

id,label


Processing cases:   0%|          | 0/50 [00:00<?, ?it/s]

[Rate Limit] Waiting 311.0 seconds...
Raw response for case 1989_75: AFTER CAREFUL CONSIDERATION OF THE ARGUMENTS PRESENTED AND THE PRINCIPLES OF NATURAL JUSTICE, I FIND THAT THE RESPONDENT'S ACTIONS WERE INDEED MISCONDUCT AND WARRANT SEVERE PUNISHMENT. THE RESPONDENT'S INVOLVEMENT IN THE FRAUDULENT ACT, PREPARATION OF BOGUS DOCUMENTS, AND SELF-AGGRANDIZEMENT DEMONSTRATE A CLEAR BREACH OF TRUST AND UNBECOMING CONDUCT FOR A GOVERNMENT SERVANT.

WHILE THE TRIBUNAL IMPOSED A LESSER PENALTY, I BELIEVE THAT SUCH LENIENCY WOULD BE DETRIMENTAL TO THE INTEGRITY OF THE ADMINISTRATION. IT IS ESSENTIAL TO MAINTAIN THE HIGHEST STANDARDS OF CONDUCT AND INTEGRITY IN PUBLIC SERVICE, AND ANY COMPROMISE ON THIS FRONT WOULD UNDERMINE THE VERY FOUNDATIONS OF OUR DEMOCRACY.

IN LIGHT OF THE PRINCIPLES DISCUSSED AND THE FACTS OF THE CASE, I HEREBY ALLOW THE APPEAL AND SET ASIDE THE ORDER OF THE TRIBUNAL. THE RESPONDENT'S ACTIONS JUSTIFY THE IMPOSITION OF A MORE SEVERE PENALTY, AND I HEREBY ORDER THAT THE O

Processing cases:   2%|▏         | 1/50 [05:20<4:21:51, 320.64s/it]

[Rate Limit] Waiting 1741.0 seconds...
Raw response for case 1959_102: AFTER CAREFUL CONSIDERATION OF THE FACTS AND THE PRINCIPLES OF NATURAL JUSTICE, I FIND THAT THE ENQUIRY HELD BY THE COMPANY ON MARCH 12 WAS INDEED FLAWED. THE COMPANY FAILED TO PROVIDE THE EMPLOYEE, DAS, WITH A FAIR OPPORTUNITY TO DEFEND HIMSELF AGAINST THE ALLEGATIONS MADE AGAINST HIM.

THE PRINCIPLES OF NATURAL JUSTICE, AS LAID DOWN IN THE CASE OF UNION OF INDIA V. T.R. VARMA, REQUIRE THAT A PARTY SHOULD HAVE THE OPPORTUNITY TO ADDUCE ALL RELEVANT EVIDENCE, THAT THE EVIDENCE OF THE OPPONENT SHOULD BE TAKEN IN HIS PRESENCE, AND THAT HE SHOULD BE GIVEN THE OPPORTUNITY TO CROSS-EXAMINE THE WITNESSES EXAMINED BY THAT PARTY.

IN THIS CASE, THE COMPANY FAILED TO EXAMINE THE WITNESSES IN THE PRESENCE OF DAS, AND DID NOT PROVIDE HIM WITH COPIES OF THEIR STATEMENTS OR READ THEM OUT TO HIM BEFORE ASKING HIM TO QUESTION THEM. THIS IS A CLEAR VIOLATION OF THE PRINCIPLES OF NATURAL JUSTICE.

FURTHERMORE, THE COMPANY HAD THE OP

...

[Rate Limit] Waiting 1654.0 seconds...
Raw response for case 1979_76: AFTER CAREFUL CONSIDERATION OF THE ARGUMENTS PRESENTED AND THE RELEVANT LAWS AND REGULATIONS, I, JUSTICE RAY, HEREBY DELIVER THE FOLLOWING VERDICT:

THE APPELLANT, A QUASI-PERMANENT ALLOTTEE, IS ENTITLED TO COMPENSATION IN CASH FOR THE LAND TAKEN OVER BY THE CENTRAL GOVERNMENT, AS PER THE DECISION RECORDED IN THE PRESS NOTE DATED AUGUST 27, 1957. THIS DECISION, WHICH WAS JOINTLY TAKEN BY THE CENTRAL GOVERNMENT AND THE PUNJAB STATE GOVERNMENT, ALLOWS FOR PAYMENT OF COMPENSATION IN CASH TO QUASI-PERMANENT ALLOTTEES WHOSE LAND, MEASURING LESS THAN TWO STANDARD ACRES, IS ACQUIRED FOR PUBLIC PURPOSES.

IN THIS CASE, THE APPELLANT'S LAND, MEASURING 1 STANDARD ACRE AND 15 1/2 UNITS, WAS TAKEN OVER BY THE CENTRAL GOVERNMENT IN JULY 1953. AS THE LAND ACQUIRED IS LESS THAN TWO STANDARD ACRES, THE APPELLANT IS ENTITLED TO COMPENSATION IN CASH, AS PER THE AFOREMENTIONED DECISION.

THE RESPONDENTS' CONTENTION THAT COMPENSATION CA

Processing cases:   6%|▌         | 3/50 [1:02:15<18:09:38, 1391.04s/it]

[Rate Limit] Waiting 1638.0 seconds...
[Rate Limit] Waiting 1236.0 seconds...
Raw response for case 1991_124: AFTER CAREFUL CONSIDERATION OF THE ARGUMENTS PRESENTED BY BOTH PARTIES AND A THOROUGH EXAMINATION OF THE RELEVANT DOCUMENTS AND RULES, I FIND THAT THE APPELLANT'S CLAIM TO BE APPOINTED AGAINST THE VACANCY ARISING LATER IS WITHOUT MERIT.

THE RULES AND REGULATIONS GOVERNING THE INDIAN POLICE SERVICE, INCLUDING THE INDIAN POLICE SERVICE CADRE RULES 1954, THE INDIAN POLICE SERVICE RECRUITMENT RULES 1954, AND THE INDIAN POLICE SERVICE APPOINTMENT BY COMPETITIVE EXAMINATION REGULATIONS 1965, DO NOT SUPPORT THE APPELLANT'S CONTENTION THAT ALL NOTIFIED VACANCIES MUST BE FILLED.

FURTHERMORE, THE AUTHORITIES' DECISION TO CLOSE THE PROCESS OF FINAL SELECTION AT A CERTAIN STAGE WAS NOT ARBITRARY, AND THE SUBSEQUENT APPOINTMENTS MADE TO OTHER SERVICES DO NOT ESTABLISH ANY ILLEGAL DISCRIMINATION AGAINST THE APPELLANT.

IN LIGHT OF THE EVIDENCE PRESENTED, I REJECT THE APPELLANT'S ARGUMENT T

Processing cases:   8%|▊         | 4/50 [1:50:18<25:18:12, 1980.28s/it]

[Rate Limit] Waiting 1541.0 seconds...
